In [1]:
import numpy as np
import pandas as pd
import altair as alt
import sklearn.linear_model as lm
from sklearn.preprocessing import add_dummy_feature
from sklearn.metrics import r2_score
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# Data Science Concepts and Analysis

### Week 7: Linear models

* Review of the simple linear model
* Multiple linear regression: linear models with many variables
* Case study: urban tree cover

## This week: multiple linear regression

**Objective**: extend the simple linear model to multiple explanatory variables.

* **Review of the simple linear model**
    + The simple linear model in matrix form
    + Estimation and uncertainty quantification
    + Parameter interpretation

* **Multiple linear regression**
    + The linear model in matrix form
    + Estimation and uncertainty quantification

* **Case study: urban tree cover**
     + Background
     + Model 1: summer temperatures, tree cover, and income
         - model fitting calculations, step by step
         - model visualization
         - interpretation of results
     + Model 2: adding population density
        - categorical variable encodings
        - results and interpretation

## Review of the simple linear model

+ The simple linear model in matrix form
+ Estimation and uncertainty quantification
+ Parameter interpretation

## The simple linear model

The **simple linear model** describes a quantitative variable $y$ as a linear function of another variable $x$ and a random error for $n$ observations:

$$y_i = \beta_0 + \beta_1 x_i + \epsilon_i \quad\begin{cases} i = 1, \dots, n \\\epsilon_i \sim N\left(0,\sigma^2\right)\end{cases}$$

* $y_i$ is the _**response variable**_
* $x_i$ is the _**explanatory variable**_
* $\epsilon_i$ is the _**error**_
* $\beta_0, \beta_1, \sigma^2$ are the _**model parameters**_
    + $\beta_0$ is the _**intercept**_
    + $\beta_1$ is the _**coefficient**_
    + $\sigma^2$ is the _**error variance**_

## Estimation

The parameters $\beta_0, \beta_1$ are estimated by **ordinary least squares** (OLS), which are best under many conditions.

The OLS estimates have a closed form:

$$\hat{\beta} = \left[\begin{array}{c}\hat{\beta}_0 \\ \hat{\beta}_1 \end{array}\right] = \left(\mathbf{X'X}\right)^{-1}\mathbf{X'y}$$

An estimate of the error variance is:

$$\hat{\sigma}^2 
    = \frac{1}{n - 2} \sum_{i = 1}^n \left(y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i\right)^2 
    = \frac{1}{n - 2}\underbrace{\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)'\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)}_\text{measure of residual variation}$$

## Uncertainty quantification

Model **uncertainty** can be thought of in terms of the _**variation in parameter estimates**_: estimates that vary a lot from sample to sample are less certain.

The variances and covariances of the estimates have a closed form:

$$\mathbf{V} = \left[\begin{array}{cc} 
    \text{var}\hat{\beta}_0 & \text{cov}\left(\hat{\beta}_0, \hat{\beta}_1\right) \\ 
    \text{cov}\left(\hat{\beta}_0, \hat{\beta}_1\right) &\text{var}\hat{\beta}_1
    \end{array}\right]
   = \sigma^2 \left(\mathbf{X'X}\right)^{-1}$$
   

This matrix is estimated by plugging in $\color{red}{\hat{\sigma}^2}$ (the estimate) for $\sigma^2$:

$$\hat{\mathbf{V}} 
    = \left[\begin{array}{cc} 
        \color{blue}{\hat{v}_{11}} & \hat{v}_{12} \\ 
        \hat{v}_{21} & \color{blue}{\hat{v}_{22}}
        \end{array}\right]
    = \color{red}{\hat{\sigma}^2}\left(\mathbf{X'X}\right)^{-1}$$

The square roots of the diagonal elements give estimated standard deviations, which are known as *standard errors*:

$$\text{SE}(\hat{\beta}_0) = \sqrt{\color{blue}{\hat{v}_{11}}} 
    \qquad\text{and}\qquad 
    \text{SE}(\hat{\beta}_1) = \sqrt{\color{blue}{\hat{v}_{22}}}$$

*For about 95% of samples, estimates will vary within about 2 standard errors.*

## Parameter interpretations

Parameter interpretations are based on the observation that under the model:

$$\mathbb{E}y_i 
    = \mathbb{E}\left(\beta_0 + \beta_1 x_i + \epsilon_i\right) 
    = \beta_0 + \beta_1 x_i + \mathbb{E}\left(\epsilon_i\right)
    = \beta_0 + \beta_1 x_i$$

* (Intercept) [at $x_i = 0$] the mean [response variable] is estimated to be [$\hat{\beta}_0$ units].

$$x_i = 0 \quad\Longrightarrow\quad \mathbb{E}y_i = \beta_0 + \beta_1 (0) = \beta_0$$

* (Slope) Every [one-unit increase in $x_i$] is associated with an estimated change in mean [response variable] of [$\hat{\beta}_1$ units].

$$\text{increment $x$} \quad\Longrightarrow\quad \beta_0 + \beta_1 (x_i + 1) = \beta_0 + \beta_1 x_i + \beta_1 = \mathbb{E}y_i + \beta_1$$

In [2]:
# import grade-aggregated seda data from hw2
seda = pd.read_csv('data/seda.csv')

# filter to math and remove NaNs
regdata = seda[seda.subject == 'math'].dropna()
regdata.head()

# save explanatory variable and response variable separately as arrays
x = add_dummy_feature(regdata[['log_income']], value = 1)
y = regdata.gap.values

# configure regression module
slr = lm.LinearRegression(fit_intercept = False)

# fit slr model
slr.fit(x, y)

# fitted values
regdata['fitted'] = slr.predict(x)

# residuals
resid = y - regdata.fitted

# residual variance
n, p = x.shape
sigmasqhat = (n - 1)*resid.var()/(n - p)

# coefficient variances/covariances
coef_vcov = np.linalg.inv(x.transpose().dot(x))*(sigmasqhat)

# coefficient standard errors
coef_se = np.sqrt(coef_vcov.diagonal())

# summary table
coef_summary = pd.DataFrame(
    {'estimate': slr.coef_,
     'standard error': coef_se},
    index = ['intercept', 'log median income']
)

# fitted std errors
regdata['fit_se'] = np.sqrt(x.dot(np.linalg.inv(x.transpose().dot(x))).dot(x.transpose()).diagonal()*sigmasqhat)

# simple scatterplot of math gap vs district income
base = alt.Chart(regdata).encode(
    x = alt.X('log_income', scale = alt.Scale(zero = False))
)

# plot
scatter = base.mark_point().encode(y = 'gap')
line = base.mark_line(color = 'red').encode(y = 'fitted')
band = base.transform_calculate(
    lwr = 'datum.fitted - 2*datum.fit_se',
    upr = 'datum.fitted + 2*datum.fit_se'
).mark_errorband(color = 'grey').encode(y = 'lwr:Q', y2 = 'upr:Q')

## Example from last time

Last time we fit a simple linear model to the SEDA math gap data. 'Fitting' the model means computing the estimates. 

In [3]:
(scatter + line + band).properties(width = 300, height = 200)

alt.LayerChart(...)

In [4]:
coef_summary

,estimate,standard error
intercept,-1.356170,0.130697
log median income,0.121057,0.011843


## Multiple linear regression
+ The linear model in matrix form
+ Estimation and uncertainty quantification

## Extending the simple linear model

The simple linear model was:

$$y_i = \beta_0 + \beta_1 x_i + \epsilon_i \quad\begin{cases} i = 1, \dots, n \\\epsilon_i \sim N\left(0,\sigma^2\right)\end{cases}
    \qquad\text{(simple linear model)}$$

It's called 'simple' because it only has a single explanatory variable $x_i$.

The **linear model** is a direct extension of the simple linear model to $p - 1$ variables $x_{i1}, \dots, x_{i, p - 1}$:

$$y_i = \beta_0 + \beta_1 x_{i1} + \cdots + \beta_{p - 1} x_{i, p - 1} + \epsilon_i \qquad\begin{cases} \epsilon_i \sim N(0, \sigma^2) \\ i = 1, \dots, n\end{cases}
    \qquad\text{(linear model)}$$

**Other names:** this is sometimes also called the *multiple regression model* or *multiple linear model*.

## The linear model in matrix form

The linear model is often written observation-wise in *indexed* form as above: $i$ indexes the observations. However, it's much more concise in matrix form:

$$\mathbf{y} = \mathbf{X}\beta + \epsilon$$

This is shorthand for:

$$\mathbf{y}: \left[\begin{array}{c} y_1 \\ y_2 \\ \vdots \\ y_n \end{array}\right] \; = \;
    \mathbf{X}: \left[\begin{array}{cccc} 
        1 &x_{11} &\cdots &x_{1, p - 1} \\
        1 &x_{21} &\cdots &x_{2, p - 1} \\
        \vdots &\vdots &\ddots &\vdots \\
        1 &x_{n1} &\cdots &x_{n, p - 1}
        \end{array}\right] \; \times \;
    \beta: \left[\begin{array}{c} \beta_0 \\ \beta_1 \\ \vdots \\ \beta_{p - 1} \end{array} \right] \; + \;
    \epsilon: \left[\begin{array}{c} \epsilon_1 \\ \epsilon_2 \\ \vdots \\ \epsilon_n \end{array}\right]$$   

Carrying out the arithmetic on the right-hand side:

$$\left[\begin{array}{c} y_1 \\ y_2 \\ \vdots \\ y_n \end{array}\right]_{\;n \times 1} \quad = \quad
    \left[\begin{array}{c} 
        \beta_0 + \beta_1 x_{11} + \cdots + \beta_{p - 1} x_{1, p - 1} + \epsilon_1 \\
        \beta_0 + \beta_1 x_{21} + \cdots + \beta_{p - 1} x_{2, p - 1} + \epsilon_2 \\
        \vdots \\
        \beta_0 + \beta_1 x_{n1} + \cdots + \beta_{p - 1} x_{n, p - 1} + \epsilon_n
        \end{array}\right]_{\;n \times 1}$$   

This is exactly the model relationship, written for each $i$.

## Good news!

Estimation and uncertainty quantification are **exactly the same as in the simple linear model**.

The OLS estimates are:

$$\hat{\beta} = \left[\begin{array}{c}\hat{\beta}_0 \\ \vdots \\ \hat{\beta}_{p - 1} \end{array}\right] = \left(\mathbf{X'X}\right)^{-1}\mathbf{X'y}$$

An estimate of the error variance is:

$$\hat{\sigma}^2 
    = \frac{1}{n - p} \sum_{i = 1}^n \left(y_i - \hat{\beta}_0 - \hat{\beta}_1 x_{i1} - \cdots - \hat{\beta}_{p - 1}x_{i, p - 1}\right)^2 
    = \frac{1}{n - p}\underbrace{\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)'\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)}_\text{measure of residual variation}$$

*The simple linear model was the special case with $p = 2$*.

## Uncertainty quantification

In the case of the multiple linear model, the variances and covariances are a $p \times p$ (instead of $2\times 2$) matrix:

$$\mathbf{V} = \left[\begin{array}{cccc} 
    \text{var}\hat{\beta}_0 
        &\text{cov}\left(\hat{\beta}_0, \hat{\beta}_1\right) 
        &\cdots &\text{cov}\left(\hat{\beta}_0, \hat{\beta}_{p - 1}\right) \\ 
    \text{cov}\left(\hat{\beta}_0, \hat{\beta}_1\right) 
        &\text{var}\hat{\beta}_1
        &\cdots &\text{cov}\left(\hat{\beta}_1, \hat{\beta}_{p - 1}\right) \\
    \vdots &\vdots &\ddots &\vdots \\
    \text{cov}\left(\hat{\beta}_0, \hat{\beta}_{p - 1}\right)
        &\text{cov}\left(\hat{\beta}_1, \hat{\beta}_{p - 1}\right)
        &\cdots
        &\text{var}\hat{\beta}_{p - 1}
    \end{array}\right]
   = \sigma^2 \left(\mathbf{X'X}\right)^{-1}$$
   

This matrix is again estimated by plugging in $\color{red}{\hat{\sigma}^2}$ (the estimate) for $\sigma^2$:

$$\hat{\mathbf{V}} 
    = \left[\begin{array}{cccc} 
        \color{blue}{\hat{v}_{11}} & \hat{v}_{12} & \cdots & \hat{v}_{1p} \\ 
        \hat{v}_{21} & \color{blue}{\hat{v}_{22}} & \cdots & \hat{v}_{2p} \\
        \vdots &\vdots &\ddots &\vdots \\
        \hat{v}_{p1} & \hat{v}_{p2} &\cdots &\color{blue}{\hat{v}_{pp}} \\
        \end{array}\right]
    = \color{red}{\hat{\sigma}^2}\left(\mathbf{X'X}\right)^{-1}$$

The square roots of the diagonal elements give *standard errors*:

$$\text{SE}(\hat{\beta}_0) = \sqrt{\color{blue}{\hat{v}_{11}}} 
    \;,\quad
    \text{SE}(\hat{\beta}_1) = \sqrt{\color{blue}{\hat{v}_{22}}} 
    \;,\quad \cdots \quad
    \text{SE}(\hat{\beta}_{p - 1}) = \sqrt{\color{blue}{\hat{v}_{pp}}}$$

## Even better news!

All the computations in `sklearn` are exactly the same.

So let's have a look at some examples.

## Case study: urban tree cover

* Background
* Model 1: summer temperatures, tree cover, and income
    + model fitting calculations, step by step
    + model visualization
    + interpretation of results
* Model 2: adding population density
    + categorical variable encodings
    + results and interpretation

## Urban tree cover data

The following are data on urban tree cover in the San Diego area:

In [5]:
np.random.seed(51421)
trees = pd.read_csv('data/utc-sd.csv').sample(n = 2000).rename(
    columns = {'fc_in_BG_g': 'tree_cover', 'IncPr': 'mean_income', 'SurfaceTem': 'mean_summer_temp'}
)

trees['income_level'] = trees.IncomeGrp.astype('category').cat.rename_categories({1: 'very low', 2: 'low', 3: 'medium', 4: 'high'})
trees['pop_density'] = trees.PopDensGrp.astype('category').cat.rename_categories({1: 'very low', 2: 'low', 3: 'medium', 4: 'high'})

trees = trees.drop(columns = ['TreeTarget', 'IncomeGrp', 'PopDensGrp'])

trees = trees[trees.mean_summer_temp > 0]

In [6]:
trees.head(4)

,Name,census_block_GEOID,tree_cover,mean_summer_temp,mean_income,income_level,pop_density
11013,"San Diego, CA",6.073010e+13,0.158259,31.986364,40951,medium,high
18997,"San Diego, CA",6.073020e+13,0.013488,34.068851,51502,high,very low
8786,"San Diego, CA",6.073010e+13,0.052844,37.098611,28454,low,low
11163,"San Diego, CA",6.073000e+13,0.217973,33.213636,40229,medium,medium


Source: McDonald RI, Biswas T, Sachar C, Housman I, Boucher TM, Balk D, et al. (2021) The tree cover and temperature disparity in US urbanized areas: Quantifying the association with income across 5,723 communities. PLoS ONE 16(4): e0249715. [doi:10.1371/journal.pone.0249715](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0249715).

## Observational units

The **observational units** are census blocks. 

Census blocks are pretty small, about the size of a city block. To get an idea of the geographic scale, here's a map of census tracts in San Francisco. Each tract contains several census blocks:

<img src = 'figures/sf-census-tracts.jpg' style = 'width:400px'>

The data comprise observations on a random sample of 1,998 census blocks in San Diego.

## Tree cover and summer temperatures

Tree canopy mitigates temperature in urban areas.

<img src = 'figures/fig1-tempcover.png' style = 'width:500px'>

In [7]:
# source code
fig1 = alt.Chart(trees).mark_circle(opacity = 0.6).encode(
    x = alt.X('tree_cover', scale = alt.Scale(type = 'pow', exponent = 1/2, zero = False)),
    y = alt.Y('mean_summer_temp', scale = alt.Scale(zero = False)),
    color = alt.Color('mean_income', scale = alt.Scale(scheme = 'redblue', reverse = True, type = 'log'))
)

# fig1.properties(width = 500, height = 350)

For this reason, urban forestry projects have been advocated as a strategy for mitigating the effects of climate change.

## Modeling objectives

Here we'll try to quantify the association between temperature, tree cover, income, and population density using multiple regression.

Let's start with a MLR model with just two explanatory variables, tree cover and income:

$$\underbrace{\text{temp}_i}_{y_i} 
    = \beta_0 + \beta_1 \underbrace{\text{cover}_i}_{x_{i1}} + \beta_2 \underbrace{\text{income}_i}_{x_{i2}} + \epsilon_i 
    \qquad i = 1, \dots, 1998$$ 

In [8]:
y = trees.mean_summer_temp.values
x_df = trees[['tree_cover', 'mean_income']]
x_df.head(3)

,tree_cover,mean_income
11013,0.158259,40951
18997,0.013488,51502
8786,0.052844,28454


## Explanatory variable matrix

We'll need to create an explanatory variable matrix of the form:

$$\mathbf{X} = \left[\begin{array}{ccc}
    1 &\text{cover}_1 &\text{income}_1 \\
    1 &\text{cover}_2 &\text{income}_2 \\
    \vdots &\vdots &\vdots \\
    1 &\text{cover}_{1998} &\text{income}_{1998} 
    \end{array}\right]$$

So we just need to add a column of ones to `x_df`:

In [9]:
# add column of ones (for intercept)
x_mx = add_dummy_feature(x_df, value = 1)

# preview
x_mx[0:3]

array([[1.00000000e+00, 1.58258747e-01, 4.09510000e+04],
       [1.00000000e+00, 1.34878570e-02, 5.15020000e+04],
       [1.00000000e+00, 5.28437900e-02, 2.84540000e+04]])

## Model fitting

Since we've added an intercept column, the fitting module should be configured **not** to fit an intercept separately.

In [10]:
# fit first model
mlr = lm.LinearRegression(fit_intercept = False)
mlr.fit(x_mx, y)
mlr.coef_

array([ 3.65559763e+01, -3.14810456e+00, -5.02075353e-05])

## Error variance estimate

Next we'll calclulate $\hat{\sigma}^2$. For this we need the fitted values:

$$\hat{\mathbf{y}} = \mathbf{X}\hat{\beta}$$

And residuals:

$$\mathbf{e} = \left(\mathbf{y} - \mathbf{X}\hat{\beta}\right) = \mathbf{y} - \hat{\mathbf{y}}$$

In [11]:
# fitted values and residuals
fitted = mlr.predict(x_mx)
resid = y - fitted

Then the error variance estimate is:

$$\hat{\sigma}^2 
= \frac{1}{n - p}\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)'\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)
= \frac{1}{n - p}\mathbf{e'e}
= \frac{n - 1}{n - p}S^2_e$$

In [12]:
# error variance estimate
n, p = x_mx.shape
sigmasqhat = resid.var()*(n - 1)/(n - p)
sigmasqhat

2.729420783462529

## Uncertainty quantification

Lastly, we'll compute the coefficient estimate standard errors:

$$\sqrt{\hat{v}_{jj}} \quad\text{from}\quad \hat{\mathbf{V}} = \hat{\sigma}^2\left(\mathbf{X'X}\right)^{-1}$$

In [13]:
xtx = x_mx.transpose().dot(x_mx) # X'X
xtxinv = np.linalg.inv(xtx) # (X'X)^{-1}
vhat = sigmasqhat*xtxinv # V
vhat

array([[ 6.72240283e-03, -6.63125605e-03, -1.25915361e-07],
       [-6.63125605e-03,  2.23049324e-01, -2.70769667e-07],
       [-1.25915361e-07, -2.70769667e-07,  3.80729809e-12]])

In [14]:
coef_se = np.sqrt(vhat.diagonal()) # (v_jj)^{1/2}
coef_se

array([8.19902606e-02, 4.72280980e-01, 1.95122989e-06])

## Quality of fit: $R^2$

There are many metrics that measure fit quality. The most common is the **proportion of variation explained by the model**, which is _**denoted by $R^2$**_:

$$R^2 
    = \frac{\text{reduction in variation}}{\text{total variation}}
    = \frac{\mathbf{\tilde{y}'\tilde{y}} - \mathbf{e'e}}{\mathbf{\tilde{y}'\tilde{y}}} 
    \qquad \text{where} \qquad
    \mathbf{\tilde{y}} = \mathbf{y} - \bar{\mathbf{y}}$$

In [15]:
# 'by hand'
y_ctr = y - y.mean()
(y_ctr.transpose().dot(y_ctr) - resid.transpose().dot(resid))/(y_ctr.transpose().dot(y_ctr))

0.30684982283103096

In [16]:
# using sklearn.metrics.r2_score
r2_score(y, fitted)

0.3068498228310309

**Interpretation**: 30% of variation in mean summer temperature is explained by tree cover and income.

## Reporting model fit

Now we've computed relevant quantities -- estimates and standard errors -- but we have yet to report these in an organized fashion. 

Typically, these are displayed in a table.

In [17]:
mlr_summary = pd.DataFrame(
    {'estimate': np.append(mlr.coef_, sigmasqhat), 
     'standard error': np.append(coef_se, float('nan'))},
    index = ['interept', 'cover', 'income', 'error variance']
)

mlr_summary

,estimate,standard error
interept,36.555976,0.081990
cover,-3.148105,0.472281
income,-0.000050,0.000002
error variance,2.729421,NaN


Along with $R^2$:

In [18]:
r2_score(y, fitted)

0.3068498228310309

## Visualization

Here are trend lines and error bands for three values of income (10th, 50th, and 90th percentiles).

<img src = 'figures/fig2-modelviz.png' style = 'width:700px'>

The error bands represent the variation of the lines -- how much they might change if we sampled different census blocks. (**Not** the variation of temperatures.)

In [19]:
# make univariate grids
tree_grid = np.linspace(trees.tree_cover.min(), trees.tree_cover.max(), 100)
income_grid = trees.mean_income.quantile([0.1, 0.5, 0.9]).values

# make mesh grid -- all combinations of univariate grid values
tx, ix = np.meshgrid(tree_grid, income_grid)
x_grid_mx = np.vstack([np.repeat(1, 300), tx.reshape(300), ix.reshape(300)]).transpose()
grid_df = pd.DataFrame(x_grid_mx, columns = ['intercept', 'tree_cover', 'mean_income'])

# add predictions and standard errors
grid_df['mean_summer_temp'] = mlr.predict(x_grid_mx)
grid_df['fit_se'] = np.sqrt(sigmasqhat*(x_grid_mx.dot(xtxinv).dot(x_grid_mx.transpose()).diagonal()))

# base layer
base = alt.Chart(grid_df).encode(
    x = alt.X('tree_cover',  scale = alt.Scale(type = 'pow', exponent = 1/2, zero = False)),
    color = alt.Color('mean_income', scale = alt.Scale(scheme = 'orangered', reverse = True, type = 'log'))
)

# regression lines
lines = base.mark_line().encode(
    y = alt.Y('mean_summer_temp', scale = alt.Scale(zero = False))
)

# uncertainty bands
bands = base.transform_calculate(
    upr = 'datum.mean_summer_temp + 2*datum.fit_se',
    lwr = 'datum.mean_summer_temp - 2*datum.fit_se'
).mark_errorband(opacity = 0.3).encode(
    y = alt.Y('lwr:Q', title = 'mean_summer_temp'),
    y2 = 'upr:Q'
)

# model visualization
fig2 = fig1.mark_point(opacity = 0.2) + bands + lines

# fig2.properties(width = 800, height = 500)

## Interpretation

So what have we learned from this exercise?

In [20]:
mlr_summary

,estimate,standard error
interept,36.555976,0.081990
cover,-3.148105,0.472281
income,-0.000050,0.000002
error variance,2.729421,NaN


* Among census blocks in San Diego, a 1% increase in tree canopy cover is associated with a decrease in average summer temperatures of 3.15 degrees Celsius, after accounting for mean income of the census block.

* Among census blocks in San Diego, a $10K increase in mean income is associated with a decrease in average summer temperatures of 0.5 degrees Celsius, after accounting for tree canopy cover.

* There's still lots of unexplained local variation in average summer temperatures; low $R^2$.

## Explaining associations

There are various mechanisms by which tree canopy reduces temperatures: providing shade; improving air quality. But what explains the weaker association between income and temperature?

Well, one possibility is that property values are higher near the ocean.

<img src = 'figures/sd-income-temp-map.png' style = 'width:600px'>

So income may simply be a proxy for distance to the ocean; and temperatures are cooler near the ocean. This is an excellent example of **confounding**!

## Confounding

In statistical jargon, we'd say that proximity to the ocean is a *confounding factor*: 

* it is correlated with higher incomes (the explanatory variable);

* it is also correlated with lower temperatures (the response);

* and so it 'explains away' the apparent association.

<img src = 'figures/confounding.svg' style = 'width:500px'>

This kind of phenomenon only affects interpretation; *we should still keep income in the model*.

## MLR with categorical variables

What if we also incorporated the population density factor (a categorical variable)?

In [21]:
trees.head(4)

,Name,census_block_GEOID,tree_cover,mean_summer_temp,mean_income,income_level,pop_density
11013,"San Diego, CA",6.073010e+13,0.158259,31.986364,40951,medium,high
18997,"San Diego, CA",6.073020e+13,0.013488,34.068851,51502,high,very low
8786,"San Diego, CA",6.073010e+13,0.052844,37.098611,28454,low,low
11163,"San Diego, CA",6.073000e+13,0.217973,33.213636,40229,medium,medium


Well, it might be natural to think we could just append this column as another variable $x_{i3}$:

$$\underbrace{\text{temp}_i}_{y_i} 
    = \beta_0 + 
        \beta_1 \underbrace{\text{cover}_i}_{x_{i1}} + 
        \beta_2 \underbrace{\text{income}_i}_{x_{i2}} + 
        \beta_3 \underbrace{\text{density}_i}_{x_{i3}} + \epsilon_i 
    \qquad i = 1, \dots, 1998$$ 
    

But this doesn't quite make sense, because the *values* of $\text{density}_i$ would be *words*! So...

$$\beta_3 \times \text{low} = \; ?$$

So we'll need to represent the categorical variable differently to include it in the model.

## Indicator variable encoding

The solution to this issue is to **encode each level** of the categorical variable using an _**indicator**_: a function whose value is zero or one to indicate a condition. 

If we want to indicate whether a census block is of low population density, we can use the indicator:

$$I(\text{density $=$ low}) = \begin{cases} 1 \text{ if population density is low} \\ 0 \text{ otherwise}\end{cases}$$

We can encode the levels of `pop_density` using a collection of indicators:

In [22]:
density_encoded = pd.get_dummies(trees.pop_density, drop_first = True)
pd.concat([trees[['pop_density']], density_encoded], axis = 1).head(4)

,pop_density,low,medium,high
11013,high,0,0,1
18997,very low,0,0,0
8786,low,1,0,0
11163,medium,0,1,0


This captures all the information about the categorical variable in quantitative terms.

## The MLR model with indicators

The model with the encoded population density variable is:

$$\underbrace{\text{temp}_i}_{y_i} 
    = \beta_0 + 
        \beta_1 \underbrace{\text{cover}_i}_{x_{i1}} + 
        \beta_2 \underbrace{\text{income}_i}_{x_{i2}} + 
        \beta_3 \underbrace{\text{low}_i}_{x_{i3}} + 
        \beta_4 \underbrace{\text{med}_i}_{x_{i4}} + 
        \beta_5 \underbrace{\text{high}_i}_{x_{i5}} + 
        \epsilon_i 
    \qquad i = 1, \dots, 1998$$ 
    

The *effect* of doing this is to allow the model to have different intercepts for each population density group.

$$\text{density $=$ very low} 
    \quad\Longrightarrow\quad 
    \mathbb{E}\text{temp}_i 
        = \underbrace{\beta_0}_\text{intercept} + \beta_1\text{cover}_i + \beta_2\text{income}_i \\
  \text{density $=$ low} 
    \quad\Longrightarrow\quad 
    \mathbb{E}\text{temp}_i 
        = \underbrace{\beta_0 + \beta_3}_\text{intercept} + \beta_1\text{cover}_i + \beta_2\text{income}_i \\
  \text{density $=$ med} 
    \quad\Longrightarrow\quad 
    \mathbb{E}\text{temp}_i 
        = \underbrace{\beta_0 + \beta_4}_\text{intercept} + \beta_1\text{cover}_i + \beta_2\text{income}_i \\
  \text{density $=$ high} 
    \quad\Longrightarrow\quad 
    \mathbb{E}\text{temp}_i 
        = \underbrace{\beta_0 + \beta_5}_\text{intercept} + \beta_1\text{cover}_i + \beta_2\text{income}_i$$

## In matrix form

The explanatory variable matrix $\mathbf{X}$ for this full model ('full' because it includes all variables) will be of the form:

$$\mathbf{X} = \left[\begin{array}{c:ccccc}
    1 &\text{cover}_{1} &\text{income}_{1} &\text{low}_1 &\text{med}_1 &\text{high}_1 \\
    1 &\text{cover}_{2} &\text{income}_{2} &\text{low}_2 &\text{med}_2 &\text{high}_2 \\
    \vdots &\vdots &\vdots &\vdots &\vdots &\vdots \\
    1 &\text{cover}_{1998} &\text{income}_{1998} &\text{low}_{1998} &\text{med}_{1998} &\text{high}_{1998} 
    \end{array}\right]$$

Here's everything to the right of the dashed partition:

In [23]:
x_full_df = pd.concat([x_df, density_encoded], axis = 1)
x_full_df.head(4)

,tree_cover,mean_income,low,medium,high
11013,0.158259,40951,0,0,1
18997,0.013488,51502,0,0,0
8786,0.052844,28454,1,0,0
11163,0.217973,40229,0,1,0


## Fit summary

The remaining calculations are all the same as before. (You can see the source code for these slides if you're interested in reviewing the details.)

Here is the model fit summary:

In [24]:
# fit model
x_full_mx = add_dummy_feature(x_full_df)
mlr_full = lm.LinearRegression(fit_intercept = False)
mlr_full.fit(x_full_mx, y)

# store dimensions of explanatory variable matrix
n, p_full = x_full_mx.shape

# compute residual variance
sigmasqhat_full = ((n - 1)/(n - p_full)) * (y - mlr_full.predict(x_full_mx)).var()

# compute standard errors
vhat_full = np.linalg.inv(x_full_mx.transpose().dot(x_full_mx)) * sigmasqhat_full
coef_full_se = np.sqrt(vhat_full.diagonal())

# construct coefficient table
mlr_full_summary = pd.DataFrame(
    data = {'estimate': np.append(mlr_full.coef_, sigmasqhat_full), 
            'standard error': np.append(coef_full_se, float('nan'))},
    index= ['intercept', 'cover', 'income', 'low density', 'medium density', 'high density', 'error variance']
)

In [25]:
# print
mlr_full_summary

,estimate,standard error
intercept,36.593462,0.114941
cover,-3.177783,0.492391
income,-0.000051,0.000002
low density,0.151946,0.095099
medium density,-0.128064,0.107888
high density,-0.149409,0.132829
error variance,2.719294,NaN


* Estimates for the cover and income coefficients are about the same.

* The population density variable changes the intercept by about $\pm 0.15$ degrees Celsius, depending on the density level.

* So the association between temperature and population density appears negligible.

## Model visualization

The estimated trends and error bands for the same three income levels and each level of population density are shown below, without the data scatter:

<img src = 'figures/fig3-fullmodel.png' style = 'width:800px'>

In [26]:
# make univariate grids
tree_grid = np.linspace(trees.tree_cover.min(), trees.tree_cover.max(), 100)
income_grid = trees.mean_income.quantile([0.1, 0.5, 0.9]).values
dens_grid = trees.pop_density.cat.categories.values

# make mesh grid -- all combinations of univariate grid values
tx, ix, dx = np.meshgrid(tree_grid, income_grid, dens_grid)
x_grid_mx = np.vstack([np.repeat(1, 1200), tx.reshape(1200), ix.reshape(1200), dx.reshape(1200)]).transpose()
grid_df = pd.DataFrame(x_grid_mx, columns = ['intercept', 'tree_cover', 'mean_income', 'pop_density'])
grid_mx = pd.concat(
    [grid_df, pd.get_dummies(grid_df.pop_density, drop_first = True)],
    axis = 1
).drop(columns = 'pop_density').astype('float64').values

# add predictions and standard errors
grid_df['mean_summer_temp'] = mlr_full.predict(grid_mx)
grid_df['fit_se'] = np.sqrt(grid_mx.dot(vhat_full).dot(grid_mx.transpose()).diagonal())
grid_df['density_order'] = grid_df.pop_density.replace({'very low': 1, 'low': 2, 'medium': 3, 'high': 4})

# base layer
base = alt.Chart(grid_df).encode(
    x = alt.X('tree_cover',  scale = alt.Scale(type = 'pow', exponent = 1/2, zero = False)),
    color = alt.Color('mean_income', scale = alt.Scale(scheme = 'orangered', reverse = True, type = 'log'))
)

# regression lines
lines = base.mark_line().encode(
    y = alt.Y('mean_summer_temp', scale = alt.Scale(zero = False)),
    strokeDash = 'pop_density'
)

# uncertainty bands
bands = base.transform_calculate(
    upr = 'datum.mean_summer_temp + 2*datum.fit_se',
    lwr = 'datum.mean_summer_temp - 2*datum.fit_se'
).mark_errorband(opacity = 0.3).encode(
    y = alt.Y('lwr:Q', title = 'mean_summer_temp'),
    y2 = 'upr:Q'
)

# model visualization
fig3 = bands + lines

# fig3.properties(
#     width = 125,
#     height = 200
# ).facet(
#     column = alt.Column('pop_density', 
#                         sort = {'field': 'density_order',
#                                 'order': 'descending'})
# )

## Comments on scope of inference

The data in this case study are from a *random sample* of census blocks in the San Diego urban area.

They are therefore representative of *all* census blocks in the San Diego urban area (population).

So the results are generliazable, meaning:
* The model approximates the *actual* associations between summer temperatures, tree cover, income, and population density in the region.

## Summary

This week focused on extending the simple linear model to multiple explanatory variables.

* The **linear model** represents a quantitative response variable $y_i$ as a linear function of several explanatory variables and a random error:

$$y_i = \beta_0 + \beta_1 x_{i1} + \cdots + \beta_{p - 1} x_{i, p - 1} + \epsilon_i$$

* When represented in matrix form $\mathbf{y} = \mathbf{X}\beta + \epsilon$, all calculations are the same as in the simple linear model.
    + OLS estimates: $\left(\mathbf{X'X}\right)^{-1}\mathbf{X'y}$
    + Error variance: $\hat{\sigma}^2 = \frac{1}{n - p}\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)'\left(\mathbf{y} - \mathbf{X}\hat{\beta}\right)$
    + Uncertainty quantification: $\hat{\mathbf{V}} = \hat{\sigma}^2\left(\mathbf{X'X}\right)^{-1}$
    

* Model fitting and interpretation was illustrated in a case study of urban tree cover in San Diego.
    + Estimation and uncertainty quantification calculations
    + Model visualization
    + Encoding categorical variables
    + Parameter interpretation